In [69]:
import pandas as pd
from scipy.stats import poisson
from sqlalchemy import and_

from models import Player, TradPlayerStats, AdvPlayerStats, Game
from data_manager import DataManager
dm = DataManager()
session = dm.get_session()


In [70]:
def estimate_probability_poisson(data, stat, n):
    mean = data[stat].mean()
    probability = 1 - poisson.cdf(n, mean)
    return probability

def american_to_decimal(american_odds):
    """Convert American odds to decimal odds."""
    if american_odds > 0:
        return 1 + (american_odds / 100)
    else:
        return 1 + (100 / abs(american_odds))

def calculate_parlay_odds(american_odds_list):
    """Calculate the combined decimal odds for a parlay given a list of American odds."""
    decimal_odds = [american_to_decimal(odds) for odds in american_odds_list]
    
    # Calculate combined odds for the parlay
    combined_odds = 1
    for odds in decimal_odds:
        combined_odds *= odds
        
    return combined_odds

# Query to join four tables with correct join conditions and multiple filters
def get_and_save_player_data(player_id, player_name):
    data = session.query(
        Player,
        TradPlayerStats,
        AdvPlayerStats,
        Game
    ).join(Game, TradPlayerStats.game_id == Game.id)\
    .join(Player, TradPlayerStats.player_id == Player.id)\
    .join(AdvPlayerStats, and_(TradPlayerStats.game_id == AdvPlayerStats.game_id, 
                                TradPlayerStats.player_id == AdvPlayerStats.player_id))\
    .filter(
        (TradPlayerStats.player_id == player_id)# &
        #  (Game.season_type == "Playoffs")
    ).all()

    # Convert the query result to a DataFrame
    data_list = []
    for player, trad_stats, adv_stats, game in data:
        row = {
            'player_name': player.name,
            'player_position': player.position,
            'minutes': trad_stats.minutes,
            'points': trad_stats.pts,
            'rebounds': trad_stats.reb,
            'assists': trad_stats.ast,
            'efg': adv_stats.efg_pct,
            'fg3a': trad_stats.fg3a,
            'fg3m': trad_stats.fg3m,
            'fg3_pct': trad_stats.fg3_pct,
            'fga': trad_stats.fga,
            'fgm': trad_stats.fgm,
            'fta': trad_stats.fta,
            'ft_pct': trad_stats.ft_pct, 
            'date': game.date,

        }
        data_list.append(row)

    data_df = pd.DataFrame(data_list)

    data_df.to_csv("data_pile/{player_name}.csv")
    return data_df


def get_player_id(player_name):
    player = session.query(Player).filter(Player.name==player_name).all()[0]
    player_id = player.id
    return player_id


def calculate_ev(probability, decimal_odds, bet_amount):
    """
    Calculate the expected value (EV) of a bet.

    Args:
        probability (float): The probability of the event occurring (between 0 and 1).
        decimal_odds (float): The decimal odds for the bet.
        bet_amount (float): The amount wagered.

    Returns:
        float: The expected value (EV) of the bet.
    """
    payout = decimal_odds * bet_amount
    ev = (probability * payout) - bet_amount
    return ev


def implied_probability(decimal_odds):
    """Convert decimal odds to implied probability."""
    return 1 / decimal_odds


def estimate_probability_poisson_under(data, stat, n):
    mean = data[stat].mean()
    probability = poisson.cdf(n, mean)  # Calculate P(X <= n)
    return probability


def get_prop_probability(player_name, stat, prop_threshold, last_n_games=25, bet_type="over"):
    player_id = get_player_id(player_name)
    data = get_and_save_player_data(player_id, player_name).sort_values(by='date', ascending=False).head(last_n_games).copy()
    
    if bet_type == "over":
        return estimate_probability_poisson(data, stat, prop_threshold)
    elif bet_type == "under":
        return estimate_probability_poisson_under(data, stat, prop_threshold)
    else:
        raise ValueError("Invalid bet type. Use 'over' or 'under'.")
    

def analyze_bet(player_name, stat, threshold, odds, bet_type):
    probability = get_prop_probability(player_name, stat, threshold, bet_type=bet_type)
    odds = american_to_decimal(odds)
    house_probability = implied_probability(odds)
    ev = calculate_ev(probability, odds, 5)
    print(f"house_probability: {house_probability}")
    print(f"our probability: {probability}")
    print(f"odds: {odds}")
    print(f"ev: {ev}")
    return probability, odds


def calculate_combined_probability(probabilities):
    """Calculate the combined probability for a parlay."""
    combined_probability = 1
    for prob in probabilities:
        combined_probability *= prob
    return combined_probability


def calculate_combined_odds(decimal_odds_list):
    """Calculate the combined decimal odds for a parlay."""
    combined_odds = 1
    for odds in decimal_odds_list:
        combined_odds *= odds
    return combined_odds


def analyze_parlay(probabilities, american_odds_list, bet_amount):
    """
    Analyze a parlay bet and print the combined probability, combined odds, and expected value.

    Args:
        probabilities (list of float): The probabilities of individual bets.
        american_odds_list (list of int): The American odds for individual bets.
        bet_amount (float): The amount wagered.
    """
    decimal_odds_list = [american_to_decimal(odds) for odds in american_odds_list]
    combined_probability = calculate_combined_probability(probabilities)
    combined_odds = calculate_combined_odds(decimal_odds_list)
    ev = calculate_ev(combined_probability, combined_odds, bet_amount)
    
    print(f"Combined Probability: {combined_probability:.4f}")
    print(f"Combined Odds: {combined_odds:.2f}")
    print(f"Expected Value (EV): ${ev:.2f}")

    return ev


In [78]:
player_name = "Jayson Tatum"
stat = "points"
threshold = 29.5
odds = -105
bet_type = "under"

analyze_bet(player_name, stat, threshold, odds, bet_type)



house_probability: 0.5121951219512195
our probability: 0.793091417691611
odds: 1.9523809523809523
ev: 2.742082886989535


(0.793091417691611, 1.9523809523809523)